# TODO:
### update frsq poi mapping
### assigning rltn tag to equal ln may not be a good idea. 

for exmaple, when the role of a way is inner(http://www.openstreetmap.org/way/48076826), meaning the the way serve a different function than rltn(http://www.openstreetmap.org/relation/381363)

it seems that "inner" role is the only fallback

examples:
- set(['cycling facilities', 'retail shop']) http://www.openstreetmap.org/way/48076826 set(['transportation']) http://www.openstreetmap.org/relation/381363
- set(['transportation']) http://www.openstreetmap.org/way/48081175 set(['outdoors and recreation']) http://www.openstreetmap.org/relation/3213622
- set(['professional service']) http://www.openstreetmap.org/way/48085321 set(['transportation']) http://www.openstreetmap.org/relation/914244
- set(['cycling facilities']) http://www.openstreetmap.org/way/48168497 set(['transportation']) http://www.openstreetmap.org/relation/383188
- set(['cycling facilities']) http://www.openstreetmap.org/way/52303179 set(['transportation']) http://www.openstreetmap.org/relation/450333
- set(['professional service']) http://www.openstreetmap.org/way/66417858 set(['outdoors and recreation']) http://www.openstreetmap.org/relation/3213622
- set(['outdoors and recreation']) http://www.openstreetmap.org/way/173918307 set(['cycling facilities']) http://www.openstreetmap.org/relation/2322204
- set(['outdoors and recreation']) http://www.openstreetmap.org/way/66418767 set(['cycling facilities']) http://www.openstreetmap.org/relation/2322204
- set(['outdoors and recreation']) http://www.openstreetmap.org/way/237887425 set(['cycling facilities']) http://www.openstreetmap.org/relation/3205766
- set(['outdoors and recreation']) http://www.openstreetmap.org/way/237887426 set(['cycling facilities']) http://www.openstreetmap.org/relation/3205766
- set(['cycling facilities']) http://www.openstreetmap.org/way/203025090 set(['transportation']) http://www.openstreetmap.org/relation/2724481
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579359 set(['transportation']) http://www.openstreetmap.org/relation/5170966
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579359 set(['transportation']) http://www.openstreetmap.org/relation/5170968
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579359 set(['transportation']) http://www.openstreetmap.org/relation/5170967
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579360 set(['transportation']) http://www.openstreetmap.org/relation/5170968
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579360 set(['transportation']) http://www.openstreetmap.org/relation/5170966
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579360 set(['transportation']) http://www.openstreetmap.org/relation/5170967
- set(['cycling facilities']) http://www.openstreetmap.org/way/296755159 set(['transportation']) http://www.openstreetmap.org/relation/2723927
- set(['cycling facilities']) http://www.openstreetmap.org/way/296794684 set(['transportation']) http://www.openstreetmap.org/relation/2723927

In [1]:
import src.constants; reload(src.constants)
from src.constants import fn_frsq_venues_dc, fn_segments_dc, epsg_dc, fn_osm_db_dc, fn_poi_frsq_dc, fn_poi_osm_dc
from src.constants import fn_mapping_for_fs, fn_mapping_for_osm

import src.geom_helper; reload(src.geom_helper)
from src.geom_helper import bfr_20m, objs_near_segs, objs_near_segs_store, gpdf_equal

import geopandas as gp
import pandas as pd

import src.poi; reload(src.poi)
from src.poi import *

# TODO

1. [x] mapped frsq venues to poi categories --> filter frsq venues by obj_near_segment
2. [x] mapped osm data to poi categories, filter osm data by obj_near_segment
3. [] remove overlap between osm and frsq
4. [] Visalization:
    - [] poi categories distribution
        1. [x] frsq venues near segments
        2. [x] osm venues near segments
        3. [] final poi near segments(after removing overlap)
    - [] poi categories spatial distribution

In [2]:
data_dir = 'data/'
path_frsq_venues_dc = data_dir + fn_frsq_venues_dc
path_osm_db_dc = data_dir + fn_osm_db_dc
path_segments_dc = data_dir + fn_segments_dc
path_mapping_for_fs = data_dir+fn_mapping_for_fs
path_mapping_for_osm = data_dir+fn_mapping_for_osm


In [3]:
frsq_venues_dc = gp.read_file(path_frsq_venues_dc)
segments_dc = gp.read_file(path_segments_dc)
poi_frsq_dc = map_frsq_venues_to_poi_category(frsq_venues_dc, path_mapping_for_fs, debug=True)
poi_frsq_near_seg_dc = objs_near_segs(poi_frsq_dc, segments_dc, bfr_20m, epsg_dc, output='objs')
print '# venues in city=',poi_frsq_dc.shape[0]
print '# venues near segments=', poi_frsq_near_seg_dc.shape[0]

objs_near_segs_store(poi_frsq_near_seg_dc, data_dir, fn_poi_frsq_dc, fn_segments_dc)

===========mapping  frsq venues to poi categories===========
venues without poi category: #venues=3139, #frsq_categories=63
top ten frsq_categories {u'': 1447, u'Building': 882, u'TV Station': 11, u'Exhibit': 51, u'Moving Target': 121, u'Zoo': 14, u'Festival': 6, u'Conference': 7, u'Intersection': 55, u'Road': 450}
# venues in city= 30187
# venues near segments= 18177
wrote obj near segments: data/poi_frsq_dc_near_segments_dc_opendc.geojson


'data/poi_frsq_dc_near_segments_dc_opendc.geojson'

In [4]:
poi_osm_dc = map_osm_to_poi_category(path_osm_db_dc, path_mapping_for_osm, debug=True)
poi_osm_near_seg_dc = objs_near_segs(poi_osm_dc, segments_dc, bfr_20m, epsg_dc, output='objs')
print '# objs have category near seg by ot:', poi_osm_near_seg_dc.ot.value_counts().to_dict()
poi_osm_pts_dc, poi_osm_ln_pls_dc = clean_and_split_poi_osm_by_geometry(poi_osm_dc)

# temp = poi_osm_near_seg_dc.copy()
# temp.category = temp.category.apply(lambda x: str(x))
# objs_near_segs_store(temp, data_dir, fn_poi_osm_dc, fn_segments_dc)
# del temp

===========mapping osm to poi category===========
# rows in table tag  = 1484405
# rows after mapping = 44060
# objs have category = 32808
# objs have category by ot: {'R': 1079, 'W': 20010, 'N': 11719}
# categories: # objs = {1: 32183, 2: 620, 3: 5}
# objs have category near seg by ot: {'R': 878, 'W': 14389, 'N': 7308}


d:\program files\anaconda2\lib\site-packages\pandas\core\generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [37]:
def remove_osm_frsq_overlap(poi_osm_pts, poi_frsq, bfr_crs, init_crs=4326, bfr_range=5):
    from src.geom_helper import crs_prepossess
    from src.osm.osmdb_filter import filter_tbtag_to_df
    osm_with_name = filter_tbtag_to_df(path_osm_db_dc, in_tags=[('name',None)])

    osm_bfr = crs_prepossess(poi_osm_pts, init_crs, bfr_crs)
    osm_bfr.geometry = osm_bfr.buffer(bfr_range)
    osm_bfr = osm_bfr.to_crs(epsg=init_crs)
    
    frsq_bfr = crs_prepossess(poi_frsq, init_crs, bfr_crs)
    frsq_bfr.geometry = frsq_bfr.buffer(bfr_range)
    frsq_bfr = frsq_bfr.to_crs(epsg=init_crs)
    
    osm_bfr_name = osm_bfr.merge(osm_with_name, how='left')
    osm_bfr_name.drop('key', axis=1, inplace=True)
    osm_bfr_name.columns = list(osm_bfr.columns) + ['name']
    osm_bfr_name.name = osm_bfr_name.name.fillna('')
    
    sjoin = gp.tools.sjoin(osm_bfr_name, frsq_bfr)
    return sjoin

In [38]:
sjoin = remove_osm_frsq_overlap(poi_osm_pts_dc, poi_frsq_dc, epsg_dc)

In [41]:
from src.utils import similarity
sjoin['name_sml'] = sjoin.apply(lambda x: similarity(x.name_left, x.name_right),axis=1)

In [ ]:
def connected_nodes(edges):
    import networkx as nx
    G = nx.Graph()
    G.add_edges_from(edges)
    return [x.nodes() for x in nx.connected_component_subgraphs(G)]
def similar_name(row,threshold=0.8):
    return row.name_sml>threshold
def no_name_but_same_category(row):
    if not row.name_sml==-1.0:
        return False
    return len(row.category_right - row.category_left)==0

In [42]:
sjoin

,ot,oid,geometry,category_left,name_left,index_right,id,name_right,category_right,name_sml
8,N,49716664,POLYGON ((-77.00900476913097 38.88750470445696...,set([cycling facilities]),,11233,4be09e4c98f2a5931cd8c25a,Office of Rep. Chris Van Hollen (MD-08),professional service,-1.000000
17,N,49718775,"POLYGON ((-76.9916004041479 38.84412699586958,...",set([cycling facilities]),,14560,5313e2d611d26c897768e425,St. Elizabeth East Gateway Pavilion,no category,-1.000000
22,N,49719415,POLYGON ((-77.05696555480179 38.90525342812081...,set([cycling facilities]),,1025,52e65ee2498e453d5ea0df67,Cafe May,no category,-1.000000
22,N,49719415,POLYGON ((-77.05696555480179 38.90525342812081...,set([cycling facilities]),,1103,5391f13f498e8a555acb7023,Ike Behar,retail shop,-1.000000
47,N,49730129,POLYGON ((-77.07697132415861 38.94316203799141...,set([cycling facilities]),,17523,4f43323319834bc91f55451c,Manny's and Olga's Pizza,food,-1.000000
58,N,49732053,POLYGON ((-77.05805425481758 38.90523442865798...,set([cycling facilities]),,1055,4bc0fb9874a9a593f019d1f6,Wells Fargo,professional service,-1.000000
64,N,49732286,"POLYGON ((-77.02800236543074 38.8920883138304,...",set([cycling facilities]),,9759,4c27ccfc97d00f4725f13eea,Michael Jackson's Fedora @Apollo Exhibit,art,-1.000000
9320,N,3016605404,POLYGON ((-77.02814476543053 38.89208861390066...,set([cycling facilities]),,9759,4c27ccfc97d00f4725f13eea,Michael Jackson's Fedora @Apollo Exhibit,art,-1.000000
72,N,49734010,POLYGON ((-77.01360948760406 38.86462800672898...,set([cycling facilities]),,14034,4de4542fd164df8575287ffe,National Response Center,professional service,-1.000000
77,N,49734612,POLYGON ((-77.03876393272729 38.93255011914005...,set([cycling facilities]),,19407,4be0a15bcb81c9b6bb7e668b,Lamont Cleaners,retail shop,-1.000000


In [30]:
poi_osm_pts_name

,ot,oid,geometry,category,name
0,N,49715946,POINT (-77.0176445 38.953198),set([cycling facilities]),
1,N,49715951,POINT (-77.01971740000001 38.9531399),set([cycling facilities]),
2,N,49715974,POINT (-77.0254044 38.9529676),set([cycling facilities]),
3,N,49716326,POINT (-77.0035556 38.8604741),set([cycling facilities]),
4,N,49716545,POINT (-77.00767500000001 38.829199),set([cycling facilities]),
5,N,49716602,POINT (-77.006002 38.824091),set([cycling facilities]),
6,N,49716610,POINT (-77.00563099999999 38.823747),set([cycling facilities]),
7,N,49716662,POINT (-77.0090656 38.8875904),set([cycling facilities]),
8,N,49716664,POINT (-77.00906240000001 38.8875047),set([cycling facilities]),
9,N,49716680,POINT (-77.0090491 38.8838384),set([cycling facilities]),


In [9]:
df

,ot,oid,key,value
0,N,49744749,name,C Street Northeast
1,N,49792620,name,Shepherd Junction
2,N,60471261,name,Cafe Saint Ex
3,N,157560663,name,Bloomingdale
4,N,157562916,name,Kingman Park
5,N,157590304,name,Forest Hills
6,N,157591605,name,Wesley Heights
7,N,157596835,name,Lanier Heights
8,N,157599737,name,Lincoln Heights
9,N,157610884,name,Columbia Heights


In [219]:
def poi_distribution(poi_frsq, poi_osm, poi=None):
    import numpy as np
    poi_frsq_distr = poi_frsq.category.value_counts().reset_index()
    poi_frsq_distr.columns = ['category','fs']
    categories, counts = np.unique(np.hstack(poi_osm.category.apply(list).apply(np.array).values), return_counts=True)
    poi_osm_distr = pd.DataFrame(zip(categories,counts),columns=['category','osm'])
    print poi_frsq_distr
    print 
    print poi_osm_distr
    print 
    print poi_frsq_distr.merge(poi_osm_distr)
    
poi_distribution(poi_frsq_near_seg_dc, poi_osm_near_seg_dc)

                   category    fs
0      professional service  3846
1                      food  2804
2               retail shop  2649
3   outdoors and recreation  2193
4               no category  1883
5            transportation  1400
6                 residence  1035
7            nightlife spot  1010
8        schools&university   699
9                       art   513
10       cycling facilities   145

                  category    osm
0                      art     49
1       cycling facilities  10695
2                     food    766
3           nightlife spot    123
4  outdoors and recreation   3529
5     professional service    793
6                residence    489
7              retail shop    962
8       schools&university    346
9           transportation   5274

                  category    fs    osm
0     professional service  3846    793
1                     food  2804    766
2              retail shop  2649    962
3  outdoors and recreation  2193   3529
4           tran